<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">Comparison of SPECFEM Synthetics
           </div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Observed vs Synthetic Data</div>
        </div>
    </div>
</div>

### **2017 CIG-LLNL Computational Seismology Workshop**


##### Authors:

* Lion Krischer ([@krischer](https://github.com/krischer))

---

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt

import numpy as np
import os

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

### Downloading Observed Data

With SPECFEM you likely simulated the 2014 South Napa earthquake in a domain encompassing the USA. If it differed for you will have to adjust this notebook slightly.

#### Event Data

Let's find the event via the USGS web service - not really necessary here but nice for visualization purposes.

In [ ]:
# Approximate coordinates of Berkeley to be able to find the event.
berkley_lon_lat = (-122.272747, 37.8715926)

In [ ]:
import obspy
from obspy.clients.fdsn import Client


client = Client("USGS")
t = obspy.UTCDateTime("2014-08-24T10:20:44.0")  # South Napa earthquake
cat = client.get_events(starttime=t - 100, endtime=t + 3600,
                        minmagnitude=6)

# Print and plot event.
print(cat)
cat.plot(projection="local", resolution="i");

#### Station Data

We also need information about the stations.

In [ ]:
# To figure out which stations were used we'll just read
# all the SPECFEM synthetics and assemble them in a list.
st_synthetics = obspy.read("SyntheticData/SPECFEM/*.sac")
stations = set((tr.stats.network, tr.stats.station) for tr in st_synthetics)
stations

In [ ]:
from obspy.clients.fdsn.header import FDSNException

# Get the inventory data for all of the stations.
c_iris = Client("IRIS")
inv = obspy.Inventory(networks=[], source="")
for net, sta in stations:
    # In these cases it is always useful to wrap
    # things in a try/except statement.
    try:
        inv += c_iris.get_stations(
            network=net,
            station=sta,
            location="00",
            channel="BH?",
            # Also request the response as we'll later use it
            # to remove the instrument response from the observed
            # data.
            level="response")
    except FDSNException:
        print("Failed to download inventory information for %s.%s" % (net, sta))
# Plot stations and event in a single plot.
fig = inv.plot(projection="local", show=False);
cat.plot(fig=fig);

#### Waveform Data

Now download the actual observed data.

In [ ]:
from obspy.clients.fdsn.header import FDSNException

event_time = cat[0].origins[0].time

st = obspy.Stream()
    
# Also download waveform data.
for net, sta in stations:
    try:
        st += c_iris.get_waveforms(
            network=net,
            station=sta,
            location="00",
            channel="BH?",
            # 5 minutes before to 30 minutes after event.
            starttime=event_time - 5 * 60,
            endtime=event_time + 30 * 60)
        print("Downloaded waveform data for %s.%s." % (net, sta))
    except FDSNException:
        print("Failed to download waveform data for %s.%s." % (net, sta))

### Interlude: STF and Rotation Helper Function




To be able to compare synthetic and observed data, they have to be as similar as possible. Observed data has been generated by some kind of source mechanism which in the far-field and for small enough earthquakes can be described by a source time function. This has to be replicated for the synthetic data. This displacement source used for the simulations is plotted here:

In [ ]:
time, stf = np.loadtxt("SyntheticData/SPECFEM/plot_source_time_function.txt").T
plt.plot(time[0:100], stf[0:100])
plt.xlabel("Time in seconds")
plt.ylabel("Displacement")
plt.show()

This is an approximate Heaviside function and very high-frequent. So we you just filter low enough we don't really have to worry about it too much. We just have to filter low enought so that the actual STF of the observed data also only has a limited influence. This varies depending on the size of the earthquake.

The horizontal components of GSN stations are often not named `E` and `N` but `1` and `2`. This might mean that they have a different orientation and have to be rotate to `E` and `N` to be able to compare them to synthetic data. This helper fucntion here does that.

In [ ]:
from obspy.signal.rotate import rotate2zne


def rotate_to_zne(st, inv):
    """
    Helper function to rotate any 3 components to ZNE -
    this will no longer be needed with the next ObsPy
    version.
    """
    # Only rotate those where its necessary.
    if set(tr.stats.channel for tr in st) != {"BH1", "BH2", "BHZ"} or len(st) != 3:
        raise Exception("Could not rotate")
        
    args = []
    for tr in st:
        args.append(tr.data)
        c = inv.select(*tr.id.split("."))[0][0][0]
        args.extend([c.azimuth, c.dip])
    # This function rotates any 3-component recording to ZNE.
    z, n, e = rotate2zne(*args)
    
    new_st = obspy.Stream()
    h = st[0].stats.copy()
    h.channel = "BHZ"
    new_st.append(obspy.Trace(z, h))
    h = st[0].stats.copy()
    h.channel = "BHN"
    new_st.append(obspy.Trace(n, h))
    h = st[0].stats.copy()
    h.channel = "BHE"
    new_st.append(obspy.Trace(e, h))
    
    return new_st

### Comparing SPECFEM Synthetics with Observed Data

In [ ]:
import os

# We'll encapsulate this in a small function to make it easier
# to try out for different stations.
def compare_data_and_synthetics(station):
    net, sta = station.split(".")
    
    st_syn = st_synthetics.select(network=net, station=sta).copy()
    st_obs = st.select(network=net, station=sta).copy()
    
    if not(st_syn):
        print("Could not find synthetics for station %s.%s." % (net, sta))
        return
    if not(st_obs):
        print("Could not find observed data for station %s.%s." % (net, sta))
        return

    st_obs.detrend("linear")
    st_obs.taper(max_percentage=0.05)

    st_obs.remove_response(
        inventory=inv, output="DISP",
        # Make sure to not touch the frequency band of interest so
        # that we don't have to apply the same spectral filter to the 
        # synthetic data.
        pre_filt=(1.0 / 400.0, 1.0 / 200.0, 2.0, 4.0))
    try:
        st_obs = rotate_to_zne(st_obs, inv)
    except:
        print("Failed to rotate %s.%s." % (net, sta))
        return

    # Also filter synthetics a bit.
    st_syn.detrend("linear")
    st_syn.taper(max_percentage=0.05)
    
    # At this point both should really be very similar.
    # Apply the same filter to both.
    (st_obs + st_syn).filter("bandpass", freqmin=1.0 / 100,
                             freqmax=1.0 / 15.0, corners=6, zerophase=True)
    
    # Force continuity - fixed in latest ObsPy master.
    for tr in st_obs:
        tr.data = np.require(tr.data, requirements=["C_CONTIGUOUS"])
    # Now be lazy and resmample to the sample points of the synthetics.
    # This should have little aliasing due to the strong filter applied before.
    st_obs.interpolate(
        sampling_rate=st_syn[0].stats.sampling_rate,
        # sinc based reconstruction filter.
        method="lanczos", a=12,
        starttime=st_syn[0].stats.starttime,
        npts=st_syn[0].stats.npts)
    
    # Get bounds for plot.
    vmax = max([np.abs(tr.data).max()
                for tr in st_obs + st_syn]) * 1.1
    
    # Plot every channel.
    for _i, c in enumerate(["Z", "N", "E"]):
        plt.subplot(3, 1, _i + 1)
        plt.title("Station %s, component %s" % (station, c))
        tr_obs = st_obs.select(component=c)[0]
        tr_syn = st_syn.select(component=c)[0]
        plt.plot(tr_obs.times(), tr_obs.data, color="black",
                 label="observed")
        plt.plot(tr_syn.times(), tr_syn.data, color="red",
                 label="synthetic")
        plt.ylim(-vmax, vmax)
        plt.ylabel("Displacement in m")
    plt.xlabel("Time since event in s")
    plt.legend()
    plt.tight_layout()
    plt.show()

for n, s in stations:
    compare_data_and_synthetics("%s.%s" % (n, s))